# Trabajo Practico, Aprendizaje No Supervisado
## Reglas de Asociacion
Grupo:
*  Martín Hunziker
*  Ramiro Caro
* Claudio Sarate 



### Carga de datos

In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import time
import pandas as pd
import logging
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [2]:
movies = pd.read_csv('movies.csv')
display(movies.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('ratings_short.csv')
display(ratings.head())

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
pelis_df = pd.merge(ratings[['userId','movieId','rating']], movies[['movieId','title', 'genres']], 
                    on='movieId', how='inner')
pelis_df=pelis_df.sort_values( by=['userId','movieId'] , axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
display (pelis_df.head(20))

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
7,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
10,1,6,4.0,Heat (1995),Action|Crime|Thriller
14,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
23,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
30,1,70,3.0,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
32,1,101,5.0,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
33,1,110,4.0,Braveheart (1995),Action|Drama|War
38,1,151,5.0,Rob Roy (1995),Action|Drama|Romance|War
39,2,318,3.0,"Shawshank Redemption, The (1994)",Crime|Drama


In [5]:
pelis=pelis_df.values[:,[0,3]]
transactions=[]
for userId, order_object in groupby(pelis, lambda x: x[0]):
    transactions.append([item[1] for item in order_object])
#display (transactions)

### Corriendo el Apriori

In [24]:
min_support = 0.001
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
logging.debug("Calculating itemset according to support...")
# time 
start_time = time.clock()
# apriori
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
# end time to calculation
end_time = time.clock()
time_apriori = (end_time-start_time)
apriori_decimals = "%.2f" % round(time_apriori,2)
print("\n\nCompletado en %s segundos\n" % apriori_decimals)

#print(frequent_itemsets) #dataframe with the itemsets

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.06)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

rules.shape


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()




Completado en 2.66 segundos



(1411132, 10)

### Reglas con confidence mayor a 0,6 ordenado y filtrado por 'confidence' y 'antecedent_len'

In [25]:
rules[rules['confidence'] > 0.6].sort_values( by=['confidence', 'lift', 'leverage','antecedent_len'], ascending=[False, False,False,True])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
32,(Aladdin (1992)),(As Good as It Gets (1997)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
33,(As Good as It Gets (1997)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
34,(Casablanca (1942)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
35,(Aladdin (1992)),(Casablanca (1942)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
36,(Dirty Dancing (1987)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
37,(Aladdin (1992)),(Dirty Dancing (1987)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
40,"(Graduate, The (1967))",(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
41,(Aladdin (1992)),"(Graduate, The (1967))",0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
42,(Mary Poppins (1964)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
43,(Aladdin (1992)),(Mary Poppins (1964)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1


### Reglas con confidence mayor a 0,6 ordenado y filtrado por 'lift' y 'antecedent_len'

In [26]:
rules[rules['confidence'] > 0.6].sort_values( by=[ 'lift', 'antecedent_len'], ascending=[False, True])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
32,(Aladdin (1992)),(As Good as It Gets (1997)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
33,(As Good as It Gets (1997)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
34,(Casablanca (1942)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
35,(Aladdin (1992)),(Casablanca (1942)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
36,(Dirty Dancing (1987)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
37,(Aladdin (1992)),(Dirty Dancing (1987)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
40,"(Graduate, The (1967))",(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
41,(Aladdin (1992)),"(Graduate, The (1967))",0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
42,(Mary Poppins (1964)),(Aladdin (1992)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1
43,(Aladdin (1992)),(Mary Poppins (1964)),0.05,0.05,0.05,1.000000,20.000000,0.0475,inf,1


### Reglas con confidence mayor a 0,6 ordenado y filtrado por 'suporte' y 'antecedent_len'

In [27]:
rules[rules['confidence'] > 0.6].sort_values( by=['support','antecedent_len'], ascending=[False, True])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
1468,"(Usual Suspects, The (1995))",(Seven (a.k.a. Se7en) (1995)),0.35,0.45,0.25,0.714286,1.587302,0.0925,1.925,1
1528,"(Usual Suspects, The (1995))",(Toy Story (1995)),0.35,0.35,0.25,0.714286,2.040816,0.1275,2.275,1
1529,(Toy Story (1995)),"(Usual Suspects, The (1995))",0.35,0.35,0.25,0.714286,2.040816,0.1275,2.275,1
315,(Braveheart (1995)),"(Usual Suspects, The (1995))",0.25,0.35,0.20,0.800000,2.285714,0.1125,3.250,1
1464,(Twelve Monkeys (a.k.a. 12 Monkeys) (1995)),(Seven (a.k.a. Se7en) (1995)),0.25,0.45,0.20,0.800000,1.777778,0.0875,2.750,1
86,(Apollo 13 (1995)),(Braveheart (1995)),0.20,0.25,0.15,0.750000,3.000000,0.1000,3.000,1
124,(Apollo 13 (1995)),(Toy Story (1995)),0.20,0.35,0.15,0.750000,2.142857,0.0800,2.600,1
129,(Apollo 13 (1995)),"(Usual Suspects, The (1995))",0.20,0.35,0.15,0.750000,2.142857,0.0800,2.600,1
172,(Babe (1995)),(Jumanji (1995)),0.20,0.25,0.15,0.750000,3.000000,0.1000,3.000,1
195,(Babe (1995)),"(Usual Suspects, The (1995))",0.20,0.35,0.15,0.750000,2.142857,0.0800,2.600,1
